<h1> Hyper-parameter tuning </h1>

**Learning Objectives**
1. Understand various approaches to hyperparameter tuning
2. Automate hyperparameter tuning using AI Platform HyperTune

## Introduction

In the previous notebook we achieved an RMSE of **4.13**. Let's see if we can improve upon that by tuning our hyperparameters.

Hyperparameters are parameters that are set *prior* to training a model, as opposed to parameters which are learned *during* training. 

These include learning rate and batch size, but also model design parameters such as type of activation function and number of hidden units.

Here are the four most common ways to finding the ideal hyperparameters:
1. Manual
2. Grid Search
3. Random Search
4. Bayesian Optimzation

**1. Manual**

Traditionally, hyperparameter tuning is a manual trial and error process. A data scientist has some intution about suitable hyperparameters which they use as a starting point, then they observe the result and use that information to try a new set of hyperparameters to try to beat the existing performance. 

Pros
- Educational, builds up your intuition as a data scientist
- Inexpensive because only one trial is conducted at a time

Cons
- Requires alot of time and patience

**2. Grid Search**

On the other extreme we can use grid search. Define a discrete set of values to try for each hyperparameter then try every possible combination. 

Pros
- Can run hundreds of trials in parallel using the cloud
- Gauranteed to find the best solution within the search space

Cons
- Expensive

**3. Random Search**

Alternatively define a range for each hyperparamter (e.g. 0-256) and sample uniformly at random from that range. 

Pros
- Can run hundreds of trials in parallel using the cloud
- Requires less trials than Grid Search to find a good solution

Cons
- Expensive (but less so than Grid Search)

**4. Bayesian Optimization**

Unlike Grid Search and Random Search, Bayesian Optimization takes into account information from  past trials to select parameters for future trials. The details of how this is done is beyond the scope of this notebook, but if you're interested you can read how it works here [here](https://cloud.google.com/blog/products/gcp/hyperparameter-tuning-cloud-machine-learning-engine-using-bayesian-optimization). 

Pros
- Picks values intelligenty based on results from past trials
- Less expensive because requires fewer trials to get a good result

Cons
- Requires sequential trials for best results, takes longer

**AI Platform HyperTune**

AI Platform HyperTune, powered by [Google Vizier](https://ai.google/research/pubs/pub46180), uses Bayesian Optimization by default, but [also supports](https://cloud.google.com/ml-engine/docs/tensorflow/hyperparameter-tuning-overview#search_algorithms) Grid Search and Random Search. 


When tuning just a few hyperparameters (say less than 4), Grid Search and Random Search work well, but when tunining several hyperparameters and the search space is large Bayesian Optimization is best.

In [1]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for AI Platform  
TFVERSION = "1.14"                # TF version for AI Platform

In [2]:
import os 
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION 

## Move code into python package

Let's package our updated code with feature engineering so it's AI Platform compatible.

In [3]:
%%bash
mkdir taxifaremodel
touch taxifaremodel/__init__.py

## Create model.py

Note that any hyperparameters we want to tune need to be exposed as command line arguments. In particular note that the number of hidden units is now a parameter.

In [4]:
%%writefile taxifaremodel/model.py
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

#1. Train and Evaluate Input Functions
CSV_COLUMN_NAMES = ["fare_amount","dayofweek","hourofday","pickuplon","pickuplat","dropofflon","dropofflat"]
CSV_DEFAULTS = [[0.0],[1],[0],[-74.0],[40.0],[-74.0],[40.7]]

def read_dataset(csv_path):
    def _parse_row(row):
        # Decode the CSV row into list of TF tensors
        fields = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)

        # Pack the result into a dictionary
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        
        # NEW: Add engineered features
        features = add_engineered_features(features)
        
        # Separate the label from the features
        label = features.pop("fare_amount") # remove label from features and store

        return features, label
    
    # Create a dataset containing the text lines.
    dataset = tf.data.Dataset.list_files(file_pattern = csv_path) # (i.e. data_file_*.csv)
    dataset = dataset.flat_map(map_func = lambda filename:tf.data.TextLineDataset(filenames = filename).skip(count = 1))

    # Parse each CSV row into correct (features,label) format for Estimator API
    dataset = dataset.map(map_func = _parse_row)
    
    return dataset

def train_input_fn(csv_path, batch_size = 128):
    #1. Convert CSV into tf.data.Dataset with (features,label) format
    dataset = read_dataset(csv_path)
      
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(buffer_size = 1000).repeat(count = None).batch(batch_size = batch_size)
   
    return dataset

def eval_input_fn(csv_path, batch_size = 128):
    #1. Convert CSV into tf.data.Dataset with (features,label) format
    dataset = read_dataset(csv_path)

    #2.Batch the examples.
    dataset = dataset.batch(batch_size = batch_size)
   
    return dataset
  
#2. Feature Engineering
# One hot encode dayofweek and hourofday
fc_dayofweek = tf.feature_column.categorical_column_with_identity(key = "dayofweek", num_buckets = 7)
fc_hourofday = tf.feature_column.categorical_column_with_identity(key = "hourofday", num_buckets = 24)

# Cross features to get combination of day and hour
fc_day_hr = tf.feature_column.crossed_column(keys = [fc_dayofweek, fc_hourofday], hash_bucket_size = 24 * 7)

# Bucketize latitudes and longitudes
NBUCKETS = 16
latbuckets = np.linspace(start = 38.0, stop = 42.0, num = NBUCKETS).tolist()
lonbuckets = np.linspace(start = -76.0, stop = -72.0, num = NBUCKETS).tolist()
fc_bucketized_plat = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "pickuplon"), boundaries = lonbuckets)
fc_bucketized_plon = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "pickuplat"), boundaries = latbuckets)
fc_bucketized_dlat = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "dropofflon"), boundaries = lonbuckets)
fc_bucketized_dlon = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "dropofflat"), boundaries = latbuckets)

def add_engineered_features(features):
    features["dayofweek"] = features["dayofweek"] - 1 # subtract one since our days of week are 1-7 instead of 0-6
    
    features["latdiff"] = features["pickuplat"] - features["dropofflat"] # East/West
    features["londiff"] = features["pickuplon"] - features["dropofflon"] # North/South
    features["euclidean_dist"] = tf.sqrt(features["latdiff"]**2 + features["londiff"]**2)

    return features

feature_cols = [
  #1. Engineered using tf.feature_column module
  tf.feature_column.indicator_column(categorical_column = fc_day_hr),
  fc_bucketized_plat,
  fc_bucketized_plon,
  fc_bucketized_dlat,
  fc_bucketized_dlon,
  #2. Engineered in input functions
  tf.feature_column.numeric_column(key = "latdiff"),
  tf.feature_column.numeric_column(key = "londiff"),
  tf.feature_column.numeric_column(key = "euclidean_dist") 
]

#3. Serving Input Receiver Function
def serving_input_receiver_fn():
    receiver_tensors = {
        'dayofweek' : tf.placeholder(dtype = tf.int32, shape = [None]), # shape is vector to allow batch of requests
        'hourofday' : tf.placeholder(dtype = tf.int32, shape = [None]),
        'pickuplon' : tf.placeholder(dtype = tf.float32, shape = [None]), 
        'pickuplat' : tf.placeholder(dtype = tf.float32, shape = [None]),
        'dropofflat' : tf.placeholder(dtype = tf.float32, shape = [None]),
        'dropofflon' : tf.placeholder(dtype = tf.float32, shape = [None]),
    }
    
    features = add_engineered_features(receiver_tensors) # 'features' is what is passed on to the model
    
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = receiver_tensors)
  
#4. Train and Evaluate
def train_and_evaluate(params):
    OUTDIR = params["output_dir"]

    model = tf.estimator.DNNRegressor(
        hidden_units = params["hidden_units"].split(","), # NEW: paramaterize architecture
        feature_columns = feature_cols, 
        model_dir = OUTDIR,
        config = tf.estimator.RunConfig(
            tf_random_seed = 1, # for reproducibility
            save_checkpoints_steps = max(100, params["train_steps"] // 10) # checkpoint every N steps
        ) 
    )

    # Add custom evaluation metric
    def my_rmse(labels, predictions):
        pred_values = tf.squeeze(input = predictions["predictions"], axis = -1)
        return {"rmse": tf.metrics.root_mean_squared_error(labels = labels, predictions = pred_values)}
    
    model = tf.contrib.estimator.add_metrics(model, my_rmse)  

    train_spec = tf.estimator.TrainSpec(
        input_fn = lambda: train_input_fn(params["train_data_path"]),
        max_steps = params["train_steps"])

    exporter = tf.estimator.FinalExporter(name = "exporter", serving_input_receiver_fn = serving_input_receiver_fn) # export SavedModel once at the end of training
    # Note: alternatively use tf.estimator.BestExporter to export at every checkpoint that has lower loss than the previous checkpoint


    eval_spec = tf.estimator.EvalSpec(
        input_fn = lambda: eval_input_fn(params["eval_data_path"]),
        steps = None,
        start_delay_secs = 1, # wait at least N seconds before first evaluation (default 120)
        throttle_secs = 1, # wait at least N seconds before each subsequent evaluation (default 600)
        exporters = exporter) # export SavedModel once at the end of training

    tf.logging.set_verbosity(v = tf.logging.INFO) # so loss is printed during training
    shutil.rmtree(path = OUTDIR, ignore_errors = True) # start fresh each time

    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

Writing taxifaremodel/model.py


## Create task.py

#### **Exercise 1**

The code cell below has two TODOs for you to complete. 

Firstly, in model.py above we set the number of hidden units in our model to be a hyperparameter. This means `hidden_units` must be exposed as a command line argument when we submit our training job to Cloud ML Engine. Modify the code below to add an flag for `hidden_units`. Be sure to include a description for the `help` field and specify the data `type` that the model should expect to receive. You can also include a `default` value. Look to the other parser arguments to make sure you have the formatting corret. 

Second, when doing hyperparameter tuning we need to make sure the output directory is different for each run, otherwise successive runs will overwrite previous runs. In `task.py` below, add some code to append the trial_id to the output direcroty of the training job. 

**Hint**: You can use `json.loads(os.environ.get('TF_CONFIG', '{}')).get('task', {}).get('trial', '')` to extract the trial id of the training job. You will want to append this quanity to the output directory `args['output_dir']` to make sure the output directory is different for each run.

In [25]:
%%writefile taxifaremodel/task.py
import argparse
import json
import os

from . import model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--hidden_units",
        help = "Hidden layer sizes to use for DNN feature columns -- provide space-separated layers",
        type = str,
        default = "10,10"
    )
    
    parser.add_argument(
        "--train_data_path",
        help = "GCS or local path to training data",
        required = True
    )
    parser.add_argument(
        "--train_steps",
        help = "Steps to run the training job for (default: 1000)",
        type = int,
        default = 1000
    )
    parser.add_argument(
        "--eval_data_path",
        help = "GCS or local path to evaluation data",
        required = True
    )
    parser.add_argument(
        "--output_dir",
        help = "GCS location to write checkpoints and export models",
        required = True
    )
    parser.add_argument(
        "--job-dir",
        help="This is not used by our model, but it is required by gcloud",
    )
    args = parser.parse_args().__dict__
    
    args["output_dir"] = os.path.join(
        args["output_dir"],
        json.loads(
            os.environ.get("TF_CONFIG", "{}")
        ).get("task", {}).get("trial", "")
    ) 
  
    # Run the training job
    model.train_and_evaluate(args)

Overwriting taxifaremodel/task.py


In [6]:
import json
json.loads(os.environ.get('TF_CONFIG', '{}')).get('task', {}).get('trial', '') 

''

In [7]:
import json
json.loads(os.environ.get('TF_CONFIG', '{}'))

{}

In [8]:
os.environ

environ{'BINARIES_PATH': '/opt/deeplearning/binaries',
        'BUCKET': 'qwiklabs-gcp-ml-49b827b781ab',
        'CLICOLOR': '1',
        'CPU_TF_SERVING_BINARIES_FOLDER': '/opt/deeplearning/binaries/tensorflow/serving/cpu',
        'DL_BIN_PATH': '/opt/deeplearning/bin',
        'DL_METADATA_PATH': '/opt/deeplearning/metadata',
        'DL_PATH': '/opt/deeplearning',
        'DL_PATH_DEPS': '/opt/deeplearning/deps',
        'ENV_VERSION_FILE_PATH': '/opt/deeplearning/metadata/env_version',
        'FRAMEWORK_FILE_PATH': '/opt/deeplearning/metadata/framework',
        'GIT_PAGER': 'cat',
        'GIT_PYTHON_REFRESH': 'quiet',
        'GPU_TF_SERVING_BINARIES_FOLDER': '/opt/deeplearning/binaries/tensorflow/serving/gpu',
        'HOME': '/home/jupyter',
        'INVOCATION_ID': '59708ea23edc4308aa64829d73ca7aa4',
        'JOURNAL_STREAM': '8:874',
        'JPY_PARENT_PID': '538',
        'JUPYTER_DEPS_PATH': '/opt/deeplearning/jupyter',
        'JUPYTER_PATH_FILE_PATH': '/opt/deeplearnin

## Create hypertuning configuration 

We specify:
1. How many trials to run (`maxTrials`) and how many of those trials can be run in parrallel (`maxParallelTrials`) 
2. Which algorithm to use (in this case `GRID_SEARCH`)
3. Which metric to optimize (`hyperparameterMetricTag`)
4. The search region in which to constrain the hyperparameter search

Full specification options [here](https://cloud.google.com/ml-engine/reference/rest/v1/projects.jobs#HyperparameterSpec).

Here we are just tuning one parameter, the number of hidden units, and we'll run all trials in parrallel. However more commonly you would tune multiple hyperparameters.

#### **Exercise 2**

Add some additional hidden units to the `hyperparam.yaml` file below to potetially explore during the hyperparameter job.

In [15]:
%%writefile hyperparam.yaml
trainingInput:
  scaleTier: BASIC
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 4
    maxParallelTrials: 2
    hyperparameterMetricTag: rmse
    enableTrialEarlyStopping: True
    algorithm: GRID_SEARCH
    params:
    - parameterName: hidden_units
      type: CATEGORICAL
      categoricalValues:
      - 10,10
      - 64,32
      - 128,64,32
      - 256,128,64,32

Overwriting hyperparam.yaml


## Run the training job|

Same as before with the addition of `--config=hyperpam.yaml` to reference the file we just created.

This will take about 20 minutes. Go to [cloud console](https://console.cloud.google.com/mlengine/jobs) and click on the job id. Once the job is completed, the choosen hyperparameters and resulting objective value (RMSE in this case) will be shown. Trials will sorted from best to worst. 

#### **Exercise 3**

Submit a hyperparameter tuning job to the cloud. Fill in the missing arguments below. This is similar to the exercise you completed in the `02_tensorlfow/g_distributed` notebook. Note that one difference here is that we now specify a `config` parameter giving the location of our .yaml file.

In [27]:
OUTDIR="gs://{}/taxifare/trained_hp_tune".format(BUCKET)
!gsutil -m rm -rf {OUTDIR}
!gcloud ai-platform jobs submit training taxifare_large_$(date -u +%y%m%d_%H%M%S) \
    --package-path=taxifaremodel \
    --module-name=taxifaremodel.task \
    --config=hyperparam.yaml \
    --job-dir=gs://{BUCKET}/taxifare \
    --python-version=3.5 \
    --runtime-version={TFVERSION} \
    --region={REGION} \
    -- \
    --train_data_path=gs://{BUCKET}/taxifare/smallinput/taxi-train.csv \
    --eval_data_path=gs://{BUCKET}/taxifare/smallinput/taxi-valid.csv  \
    --train_steps=5000 \
    --output_dir={OUTDIR}

CommandException: 1 files/objects could not be removed.
Job [taxifare_large_191126_040156] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe taxifare_large_191126_040156

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs taxifare_large_191126_040156
jobId: taxifare_large_191126_040156
state: QUEUED


## Results

The best result is RMSE **4.02** with hidden units = 128,64,32. 

This improvement is modest, but now that we have our hidden units tuned let's run on our larger dataset to see if it helps. 

Note the passing of hyperparameter values via command line

In [ ]:
gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned


In [26]:
OUTDIR="gs://{}/taxifare/trained_large_tuned".format(BUCKET)
!gsutil -m rm -rf {OUTDIR} # start fresh each time
!gcloud ai-platform local train \
    --package-path=taxifaremodel \
    --module-name=taxifaremodel.task \
    -- \
    --train_data_path=gs://cloud-training-demos/taxifare/large/taxi-train*.csv \
    --eval_data_path=gs://cloud-training-demos/taxifare/small/taxi-valid.csv  \
    --train_steps=2000 \
    --output_dir={OUTDIR} \
    --hidden_units="128,64,32"
            

CommandException: 1 files/objects could not be removed.
2019-11-26 03:57:44.761310: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
1.15.0
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 200 or save_checkpoints_secs None.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

In [28]:
OUTDIR="gs://{}/taxifare/trained_large_tuned".format(BUCKET)
!gsutil -m rm -rf {OUTDIR} # start fresh each time
!gcloud ai-platform jobs submit training taxifare_large_$(date -u +%y%m%d_%H%M%S) \
    --package-path=taxifaremodel \
    --module-name=taxifaremodel.task \
    --job-dir=gs://{BUCKET}/taxifare \
    --python-version=3.5 \
    --runtime-version={TFVERSION} \
    --region={REGION} \
    --scale-tier=STANDARD_1 \
    -- \
    --train_data_path=gs://cloud-training-demos/taxifare/large/taxi-train*.csv \
    --eval_data_path=gs://cloud-training-demos/taxifare/small/taxi-valid.csv  \
    --train_steps=200000 \
    --output_dir={OUTDIR} \
    --hidden_units="128,64,32"

Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/#1574740669646522...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/checkpoint#1574740889029516...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/eval/#1574740706465079...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/eval/events.out.tfevents.1574740706.tensorflow-20191118-153532#1574740899225858...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/events.out.tfevents.1574740669.tensorflow-20191118-153532#1574740914460739...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/export/exporter/1574740899/variables/variables.data-00000-of-00002#1574740913073378...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/export/exporter/#1574740901248359...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/taxifare/trained_large_tuned/export/exporter/1574740899/variables/variables.data-00001-of-

## Analysis

Our RMSE improved to **3.85**

## Challenge excercise

Try to beat the current RMSE:

- Try adding more engineered features or modifying existing ones
- Try tuning additional hyperparameters 

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License